In [ ]:
!git clone https://github.com/vslvskyy/tts_fastspeech2
%cd tts_fastspeech2

!pip install -r requirements.txt

# download Waveglow
!wget --load-cookies /tmp/cookies.txt \
"https://docs.google.com/uc?export=download&confirm= \
$(wget --quiet --save-cookies /tmp/cookies.txt \
--keep-session-cookies --no-check-certificate \
'https://docs.google.com/uc?export=download&id=1MDLbGuUDNEs8p96fc6IPJYwqv1-1hbrM' \
-O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1MDLbGuUDNEs8p96fc6IPJYwqv1-1hbrM" \
-O waveglow_256channels_ljs_v2.pt \
&& rm -rf /tmp/cookies.txt

!mkdir -p waveglow/pretrained_model/
!mv waveglow_256channels_ljs_v2.pt waveglow/pretrained_model/waveglow_256channels.pt

# download fastspeech2 model
!wget --load-cookies /tmp/cookies.txt \
"https://docs.google.com/uc?export=download&confirm= \
$(wget --quiet --save-cookies /tmp/cookies.txt \
--keep-session-cookies --no-check-certificate \
'https://docs.google.com/uc?export=download&id=11UH13WOhwAcagACaARpr_mnqIiQQrXCJ' \
-O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=11UH13WOhwAcagACaARpr_mnqIiQQrXCJ" \
-O checkpoint_117000.pth.tar \
&& rm -rf /tmp/cookies.txt

In [ ]:
import torch
from model import FastSpeech2
from utils import get_WaveGlow
from configs import MelSpectrogramConfig, FastSpeechConfig, TrainConfig



mel_config = MelSpectrogramConfig()
model_config = FastSpeechConfig()
train_config = TrainConfig()

WaveGlow = get_WaveGlow()
WaveGlow = WaveGlow.cuda()

model = FastSpeech2(model_config, train_config, mel_config)
model = model.to(train_config.device)

model.load_state_dict(torch.load('./checkpoint_117000.pth.tar', map_location='cuda:0')['model'])
model = model.eval()

In [44]:
from synthesis import synthesize, get_data
from waveglow import inference


inp = [
    "A defibrillator is a device that gives a high energy electric shock to the heart of someone who is in cardiac arrest",
    "Massachusetts Institute of Technology may be best known for its math, science and engineering education",
    "Wasserstein distance or Kantorovich Rubinstein metric is a distance function defined between probability distributions on a given metric space"
]
inp = get_data(inp, train_config)

!mkdir results
for speed in [0.8, 1.2]:
    pitch, energy = 1., 1.

    for i in range(len(inp)):
        mel_spec = synthesize(
            model, inp[i], train_config.device, speed, pitch, energy
        )
        inference(
            mel_spec, WaveGlow,
            f"./results/s={speed}_p={pitch}_e={energy}_{i}_waveglow.wav"
        )

for pitch in [0.8, 1.2]:
    speed, energy = 1., 1.

    for i in range(len(inp)):
        mel_spec = synthesize(
            model, inp[i], train_config.device, speed, pitch, energy
        )
        inference(
            mel_spec, WaveGlow,
            f"./results/s={speed}_p={pitch}_e={energy}_{i}_waveglow.wav"
        )

for energy in [0.8, 1.2]:
    speed, pitch = 1., 1.

    for i in range(len(inp)):
        mel_spec = synthesize(
            model, inp[i], train_config.device, speed, pitch, energy
        )
        inference(
            mel_spec, WaveGlow,
            f"./results/s={speed}_p={pitch}_e={energy}_{i}_waveglow.wav"
        )

for coef in [0.8, 1., 1.2]:
    speed, pitch, energy = coef, coef, coef

    for i in range(len(inp)):
        mel_spec = synthesize(
            model, inp[i], train_config.device, speed, pitch, energy
        )
        inference(
            mel_spec, WaveGlow,
            f"./results/s={speed}_p={pitch}_e={energy}_{i}_waveglow.wav"
        )

In [45]:
# set i, speed, pitch and energy you need

from IPython import display

i = 1
speed, pitch, energy = 1., 1., 1.
display.Audio(f"./results/s={speed}_p={pitch}_e={energy}_{i}_waveglow.wav")